### OpenAI SDK Agents

El SDK Agents es una herramienta que permite crear agentes de IA personalizados utilizando los modelos de OpenAI. Algunos proveedores de modelos de lenguaje como Google Gemini pueden usar esta API con soporte limitado.

SDK Agents tiene pocas abstracciones y es fácil de usar. El Agents SDK tiene un conjunto muy pequeño de primitivas:

* Agentes (`Agents`), que son LLMs equipados con instrucciones y herramientas.
* Delegaciones (`Handoffs`), que permiten a los agentes delegar tareas específicas a otros agentes.
* Barreras de seguridad (`Guardrails`), que permiten validar las entradas y salidas de los agentes.

Además, el SDK viene con trazabilidad incorporada que te permite visualizar y depurar tus flujos de agentes, así como evaluarlos e incluso ajustar modelos para tu aplicación.

En esta práctica vamos a utilizar el SDK Agents para crear una aplicación que genere un informe profesional a partir de una pregunta formulada por el usuario. Varios agentes se coordinarán para realizar una búsqueda en Internet y componer un informe que será enviado al usuario.

Realizamos las importacines necesarias

In [ ]:
from openai import AsyncOpenAI
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool, OpenAIChatCompletionsModel
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import os
from typing import Dict
from IPython.display import display, Markdown
import requests
from google import genai
from google.genai.types import Tool, GoogleSearch, GenerateContentConfig
import json
import gradio as gr

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


### `Searcher Agent`

Creamos el primer agente que será capaz de buscar información en Internet. Este agente realiza la búsqueda usando Google Gemini. Para hacer búsquedas en Internet con el SDK Agents hay que usar la clase `WebSearchAgent`. Pero esta clase no es compatible con `OpenAIChatCompletionsModel`. `OpenAIChatCompletionsModel` es una función que se debe usar cuando se crean agentes con un proveedor distinto de OpenAI. Por lo tanto, se ha tenido que utilizar otra estrategia para poder realizar búsquedas en Internet. Se ha realizado usando la API de Google Gemini llamada Google GenAI. Los pasos son los siguientes:

* 1 Importamos la clave de Google Gemini.
* 2 Creamos el modelo que se usará para crear los agentes. Normalmente este modelo sería una simple cadena de texto con el nombre del modelo. Pero cuando el modelo no es de OpenAI el proceso es un poco más complicado.
* 3 Creamos el cliente que se conectará a la API de Google GenAI.
* 4 Creamos la `tool` de GenAI que se usará para realizar las búsquedas en Internet.
* 5 Creamos una función de búsqueda que utiliza la `tool`.
* 6 ...

... No hemos terminado de crear el agente, pero ya tenemos la función de búsqueda que se usará en el agente y podemos probarla.

In [12]:
# 1.- Importamos la clave de Google Gemini.
load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')

if google_api_key:
    print(f"La clave de API de Google existe y comienza con {google_api_key[:2]}")
else:
    print("La clave de API de Google no está configurada (y esto no es opcional)")

GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"


# 2.- Creamos el modelo que se usará para crear los agentes.
chat_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
chat_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=chat_client)

# 3.- Creamos el cliente que se conectará a la API de Google GenAI.
search_client = genai.Client(api_key=google_api_key)

# 4.- Creamos la `tool` de GenAI que se usará para realizar las búsquedas en Internet.
INSTRUCTIONS = "Eres un asistente de investigación. Dado un término de búsqueda, buscas en la web ese término y \
produces un resumen conciso de los resultados. El resumen debe tener 2-3 párrafos y menos de 300 \
palabras. Captura los puntos principales. Escribe de forma sucinta, no es necesario usar oraciones completas ni buena \
gramática. Esto será utilizado por alguien que está sintetizando un informe, así que es vital que captures la \
esencia y ignores cualquier información irrelevante. No incluyas ningún comentario adicional aparte del propio resumen."

search_tool = Tool(google_search=GoogleSearch())

config = GenerateContentConfig(
    system_instruction=INSTRUCTIONS,
    tools=[search_tool],
    response_modalities=["TEXT"]
)

# 5.- Creamos una función de búsqueda que utiliza la `tool`.
def search_term(term: str) -> Dict[str, str]:
    """Realiza una búsqueda en Google y devuelve los resultados."""
    response = search_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=term,
        config=config
    )
    return response

La clave de API de Google existe y comienza con AI


In [13]:
response = search_term("Día y hora actual en Madrid (España) y previsión del tiempo para los próximos 3 días.")
display(Markdown(response.text))

La fecha y hora actuales en Madrid, España, son las 12:34 PM CEST del miércoles 28 de mayo de 2025.

El pronóstico del tiempo para los próximos 3 días es:

*   **Jueves:** Soleado, con una temperatura máxima de 95°F (34°C) y una mínima de 67°F (19°C).
*   **Viernes:** Parcialmente nublado, con una temperatura máxima de 92°F (33°C) y una mínima de 66°F (19°C).
*   **Sábado:** Parcialmente nublado, con una temperatura máxima de 92°F (33°C) y una mínima de 68°F (20°C).

... continuamos:

* 6 Convertimos la función de búsqueda en una `tool` que se usará en el agente. Para ello usamos el decorador `@function_tool`.
* 7 ...

In [14]:
# 6.- Convertimos la función de búsqueda en una `tool
@function_tool
def search_term(term: str) -> Dict[str, str]:
    """Realiza una búsqueda en Google y devuelve los resultados."""
    response = search_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=term,
        config=config
    )
    return response

In [14]:
# search_term ya no es una función normal de Python sino una tool que se puede usar en un agente.
search_term

FunctionTool(name='search_term', description='Realiza una búsqueda en Google y devuelve los resultados.', params_json_schema={'properties': {'term': {'title': 'Term', 'type': 'string'}}, 'required': ['term'], 'title': 'search_term_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f632a47d940>, strict_json_schema=True)

* 7 Creamos un agente que usa la `tool` de búsqueda. Con `ModelSettings` forzamos al agente a que use la `tool` siempre.

In [15]:
# 7.- Creamos un agente que usa la `tool` de búsqueda
search_agent = Agent(
    name="Agente de Búsqueda",
    instructions=INSTRUCTIONS,
    tools=[search_term],
    model=chat_model,
    model_settings=ModelSettings(tool_choice="required"),
)

* 8 Probamos el agente. Está metido en un contexto `trace` que permite trazabilidad. La trazabilidad es muy fácil de implementar con OpenAI Agents SDK pero no he sido capaz de configurarlo en Google Gemini.

In [17]:
message = "Frameworks de agentes de IA nuevos y más populares en el año actual"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

AI agent frameworks are gaining traction in 2024, simplifying the development of AI systems capable of reasoning, planning, and acting autonomously. These frameworks offer pre-packaged tools and features, enabling developers to build sophisticated AI agents for complex tasks, including reasoning, planning, action capability (using external tools), and memory of past interactions.

Popular frameworks include LangChain, known for its modular architecture; AutoGen, designed for multi-agent AI systems; CrewAI, which facilitates the creation of collaborative AI agents; LlamaIndex Workflows, focusing on asynchronous operations; and Semantic Kernel, which helps developers embed AI capabilities into existing applications. These frameworks offer simplified LLM integration, pre-built algorithms, customizable models, and tools for building agent workflows. The choice of framework depends on factors like task complexity, team skill level, and integration needs.

### `Planner Agent`

Este agente, dado un término de búsqueda, propone un conjunto de preguntas que se pueden hacer para afinar la búsqueda. En la construcción del agente se utiliza el parámetro `output_type` para especificar el formato de salida esperado. A esto se le llama salidas estructuradas. En este caso, se espera una lista de preguntas, donde cada pregunta tiene los campos `reason` y `query`. Observe que el esquema se define usando `Pydantic` y es el agente el que tiene que entender el esquemea y devolver la salida en el formato correcto.

In [27]:
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"Eres un asistente de investigación útil. Dada una consulta, genera un conjunto de búsquedas web en español\
para responder mejor a la pregunta. Devuelve {HOW_MANY_SEARCHES} términos de búsqueda a consultar."

# Usamos Pydantic para definir el esquema de respuesta - conocido como "Salidas Estructuradas"
class WebSearchItem(BaseModel):
    reason: str = Field(description="Tu razonamiento sobre por qué esta búsqueda es importante para la consulta.")
    
    query: str = Field(description="El término de búsqueda a utilizar para la búsqueda web.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="Una lista de búsquedas web para realizar y responder mejor a la consulta.")

# Este agente no hace búsquedas web directamente, sino que genera un plan de búsqueda con tres preguntas.
planner_agent = Agent(
    name="Agente Planificador",
    instructions=INSTRUCTIONS,
    model=chat_model,
    output_type=WebSearchPlan,
)

Probamos el agente. Observe que propone tres preguntas con el esquema definido.

In [21]:
message = "Frameworks de agentes de IA nuevos y más populares en el año actual"

with trace("Plan"):
    result = await Runner.run(planner_agent, message)

result.final_output

WebSearchPlan(searches=[WebSearchItem(reason='To identify recently developed or trending AI agent frameworks.', query='new AI agent frameworks 2024'), WebSearchItem(reason='To find frameworks that are widely adopted and well-regarded in the AI community.', query='popular AI agent frameworks'), WebSearchItem(reason='To discover cutting-edge research and development in AI agent technology.', query='advanced AI agent frameworks')])

### Agente `Emailer Agent`

Este agente es capaz de enviar correos electrónicos. Pare ello creamos la `tool` `send_email` que se encargará de enviar el correo electrónico utilizando la API de Mailgun.

In [ ]:
@function_tool
def send_email(subject: str, html_body: str, to: str, name: str = None) -> Dict[str, str]:
    """Enviar un correo electrónico"""
    from_email = os.getenv('MAILGUN_FROM')
    to_email = f"{name} <{to}>" if name else to
    content = html_body

    requests.post(
  		f"https://api.mailgun.net/v3/{os.getenv('MAILGUN_SANDBOX')}/messages",
  		auth=("api", os.getenv('MAILGUN_API_KEY')),
  		data={"from": from_email,
			"to": to_email,
  			"subject": subject,
  			"html": content})

    return {"status": "éxito"}

In [18]:
INSTRUCTIONS = """Eres capaz de enviar un correo electrónico en HTML bien formateado a partir de un informe detallado.
Se te proporcionará un informe detallado. Debes usar tu herramienta para enviar un solo correo, convirtiendo el informe
en un HTML limpio y bien presentado, con una línea de asunto adecuada."""

email_agent = Agent(
    name="Agente de correo",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model=chat_model,
    model_settings=ModelSettings(tool_choice="required")
)

Probamos el agente. Revisar la carpeta de `spam` porque suele llegar allí.

In [55]:
with trace("Email"):
    params = json.dumps({
        "subject": "Informe de investigación sobre frameworks de agentes de IA",
        "html_body": "bla **bla** bla",
        "to": "surtich@gmail.com",
        "name": "Javier"
    })
    result = await Runner.run(email_agent, params)

### `Writer Agent`

Este agente es capaz de redactar un informe profesional a partir de un conjunto de preguntas y respuestas.

In [19]:
INSTRUCTIONS = (
    "Eres un investigador senior encargado de redactar un informe cohesivo para una consulta de investigación. "
    "Se te proporcionará la consulta original y una investigación inicial realizada por un asistente de investigación.\n"
    "Primero debes elaborar un esquema para el informe que describa la estructura y el flujo del mismo. Luego, genera el informe y devuélvelo como salida final.\n"
    "La salida final debe estar en formato markdown y debe ser extensa y detallada. Apunta a 5-10 páginas de contenido, al menos 1000 palabras."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="Un resumen breve de 2-3 frases sobre los hallazgos.")

    markdown_report: str = Field(description="El informe final")

    follow_up_questions: list[str] = Field(description="Temas sugeridos para investigar más a fondo")


writer_agent = Agent(
    name="AgenteRedactor",
    instructions=INSTRUCTIONS,
    model=chat_model,
    output_type=ReportData,
)

### Funciones auxiliares

Las siguientes funciones permiten conectar los agentes entre sí. El SDK Agents es de bajo nivel y este proceso hay que hacerlo manualmente.

La `plan_searches` utilza el planificador para generar un conjunto de preguntas a partir de un término de búsqueda.

In [20]:
async def plan_searches(query: str) -> WebSearchPlan:
    """ Utiliza el planner_agent para planificar qué búsquedas realizar para la consulta """
    print("Planificando búsquedas...")
    result = await Runner.run(planner_agent, f"Consulta: {query}")
    print(f"Se realizarán {len(result.final_output.searches)} búsquedas")
    return result.final_output_as(WebSearchPlan) # no es estrictamente necesario. Se puede ejecutar simplemente result.final_output

Observe que la función `plan_searches` es una corutina.

In [20]:
query = "Informe sobre institutos de la Comunidad de Madrid que imparten DAW"
plan_searches(query)

<coroutine object plan_searches at 0x7f3dabf55d20>

Para ejecutala hay que usar:

* `asyncio.run(plan_searches("Python programming language"))` si estamos en un programa de Python.
* `await plan_searches("Python programming language")` si estamos en un entorno asíncrono como Jupyter Notebook.

In [ ]:
search_plan = await plan_searches(query)
search_plan

Planificando búsquedas...
Se realizarán 3 búsquedas


WebSearchPlan(searches=[WebSearchItem(reason='Busca directamente institutos en la Comunidad de Madrid que ofrezcan el ciclo formativo de Desarrollo de Aplicaciones Web (DAW).', query='institutos comunidad madrid ciclo DAW'), WebSearchItem(reason='Busca listados o directorios de centros formativos que imparten DAW en la Comunidad de Madrid.', query='listado centros formativos DAW Madrid'), WebSearchItem(reason='Aunque la consulta principal es sobre los institutos, conocer los requisitos de acceso puede ayudar a identificar los centros que ofrecen la formación.', query='requisitos acceso ciclo DAW Madrid')])

La función `perform_searches` utiliza la función `search` para realizar las búsquedas en Internet.

In [21]:
async def perform_searches(search_plan: WebSearchPlan):
    """ Llama a search() para cada elemento en el plan de búsqueda """
    print("Buscando...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches] # Ejecuta en paralelo las búsquedas
    num_completed = 0
    results = []
    for task in asyncio.as_completed(tasks):
        result = await task
        if result is not None:
            results.append(result)
        num_completed += 1
        print(f"Buscando... {num_completed}/{len(tasks)} completado")
    print("Búsquedas finalizadas")
    return results

async def search(item: WebSearchItem):
    """ Utiliza el search_agent para realizar una búsqueda web por cada elemento del plan de búsqueda """
    input = f"Término de búsqueda: {item.query}\nRazón para buscar: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

Probamos

In [ ]:
search_results = await perform_searches(search_plan)
search_results

Buscando...
Buscando... 1/3 completado
Buscando... 2/3 completado
Buscando... 3/3 completado
Búsquedas finalizadas


['Para acceder al ciclo de Grado Superior en Desarrollo de Aplicaciones Web (DAW) en Madrid, es necesario cumplir con alguno de los siguientes requisitos:\n\n*   Poseer el título de Bachiller (LOE o LOGSE), BUP, COU o una acreditación equivalente o superior.\n*   Tener un título de Técnico Superior, Técnico Especialista o equivalente a efectos académicos.\n*   Haber superado el segundo curso de cualquier modalidad de Bachillerato Experimental.\n*   Estar en posesión de un título universitario o equivalente.\n*   Haber superado la prueba de acceso a la universidad para mayores de 25 años.\n*   Superar la prueba de acceso a ciclos formativos de grado superior (tener al menos 19 años, o 18 si se posee un título de Técnico relacionado).\n\nNo se exige una nota mínima específica para el acceso, pero es indispensable cumplir con alguna de las condiciones académicas mencionadas. Algunos centros pueden requerir superar una prueba de admisión.',
 'Varios centros en Madrid ofrecen el ciclo forma

La siguiente función escribe el informe a partir de las preguntas y respuestas obtenidas.

In [22]:
async def write_report(query: str, search_results: list[str]):
    """ Utiliza el agente redactor para escribir un informe basado en los resultados de búsqueda """
    print("Pensando en el informe...")
    input = f"Consulta original: {query}\nResultados de búsqueda resumidos: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Informe terminado")
    return result.final_output

Probamos

In [24]:
await write_report(query, search_results)

Pensando en el informe...
Informe terminado


ReportData(short_summary='Este informe proporciona una visión general de los institutos en la Comunidad de Madrid que ofrecen el ciclo formativo de Grado Superior en Desarrollo de Aplicaciones Web (DAW).  Se describen los aspectos clave del ciclo formativo, se listan algunos de los centros que lo imparten y se proponen criterios para su comparación.  El informe sirve como punto de partida para estudiantes interesados en esta área, ofreciendo información inicial para tomar decisiones informadas.', markdown_report='# Informe sobre Institutos de la Comunidad de Madrid que imparten DAW (Desarrollo de Aplicaciones Web)\n\n## Esquema del Informe\n\n1.  **Introducción**\n    *   1.1 Propósito del Informe\n    *   1.2 Alcance\n    *   1.3 Metodología\n\n2.  **¿Qué es DAW?**\n    *   2.1 Descripción general del ciclo formativo de Grado Superior en Desarrollo de Aplicaciones Web (DAW)\n    *   2.2 Competencias profesionales que se adquieren\n    *   2.3 Salidas profesionales\n\n3.  **Institutos 

Por último, escribimos la función que envíe el correo electrónico con el informe generado.

In [23]:
async def send_report(report: ReportData, to: str, name: str = None):
    """ Utiliza el agente de correo para enviar un correo con el informe """
    print("Redactando correo...")
    send_email_params = {
        "html_body": report.markdown_report,
        "to": to,
        "name": name
    }
    if (name):
        send_email_params["name"] = name
    result = await Runner.run(email_agent, json.dumps(send_email_params))
    print("Correo enviado")
    return report

Probamos

In [37]:
report = ReportData(
    short_summary="Resumen breve de los hallazgos del informe.",
    markdown_report="Informe sobre institutos de la Comunidad de Madrid que imparten DAW\n\nEste es un informe detallado...",
    follow_up_questions=["¿Qué otros institutos ofrecen DAW?", "¿Cuáles son las tasas de empleo de los graduados?"]
)
await send_report(report, "surtich@gmail.com")

Redactando correo...
Correo enviado


ReportData(short_summary='Resumen breve de los hallazgos del informe.', markdown_report='Informe sobre institutos de la Comunidad de Madrid que imparten DAW\n\nEste es un informe detallado...', follow_up_questions=['¿Qué otros institutos ofrecen DAW?', '¿Cuáles son las tasas de empleo de los graduados?'])

### Prueba de la aplicación

Finalmente, probamos la aplicación completa. Creamos función que coordina todos los agentes anteriores y envía el informe por correo electrónico.

In [24]:
query = "Aplicaciones del lenguaje Python para Administradores de Sistemas en 2025"
to = "surtich@gmail.com"

with trace("Rastreo de investigación"):
    print("Iniciando investigación...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_report(report, to)
    print("¡Hurra!")

Iniciando investigación...
Planificando búsquedas...
Se realizarán 3 búsquedas
Buscando...
Buscando... 1/3 completado
Buscando... 2/3 completado
Buscando... 3/3 completado
Búsquedas finalizadas
Pensando en el informe...
Informe terminado
Redactando correo...
Correo enviado
¡Hurra!


### Integración con Gradio

En esta caso no podemos usar la función `ChatInterface` de Gradio porque necesitamos recoger la entrada del usuario. Creamos una función `run` y se la asociamos a la función `launch`. La función `run` emite (`yield`) valores que Gradio mostrará en la interfaz.



In [28]:
async def run(query: str, to: str, name: str = None):
    """ Ejecuta el proceso de investigación profunda, generando actualizaciones de estado y el informe final """
    print("Iniciando investigación...")
    search_plan = await plan_searches(query)
    yield "Búsquedas planificadas, iniciando búsqueda..."     
    search_results = await perform_searches(search_plan)
    yield "Búsquedas completadas, redactando informe..."
    report = await write_report(query, search_results)
    yield "Informe redactado, enviando correo..."
    await send_report(report, to, name)
    yield "Correo enviado, investigación completada"
    yield report.markdown_report

In [ ]:
with gr.Blocks(theme=gr.themes.Default(primary_hue="sky")) as ui:
    gr.Markdown("# Investigación Profunda")
    query_textbox = gr.Textbox(label="¿Sobre qué tema te gustaría investigar?")
    query_email = gr.Textbox(label="¿Cuál es tu correo electrónico?")
    query_name = gr.Textbox(label="¿Cuál es tu nombre? (opcional)")
    run_button = gr.Button("Ejecutar", variant="primary")
    report = gr.Markdown(label="Informe")
    
    run_button.click(fn=run, inputs=[query_textbox, query_email, query_name], outputs=report)
    query_textbox.submit(fn=run, inputs=query_textbox, outputs=report)

ui.launch(inbrowser=True)

/home/surtich/projects/IA-para-desarroladores/.venv/lib/python3.12/site-packages/gradio/utils.py:1032: UserWarning: Expected at least 2 arguments for function <function run at 0x7fe6c8c81080>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Iniciando investigación...
Planificando búsquedas...
Se realizarán 3 búsquedas
Buscando...
